# EMSN 2.0 - Vocalization Classifier Training Batch 2

Train de **92 ontbrekende soorten** met GPU acceleratie.

## Deze batch bevat:
- 3 Prioriteit 1 soorten (Kauw, Kokmeeuw, Nijlgans)
- 47 Prioriteit 2 soorten
- 42 Prioriteit 3 soorten

**Geschatte tijd:** 2-4 uur op Colab GPU

In [ ]:
# Check GPU
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Install dependencies
!pip install librosa scikit-learn scikit-image matplotlib tqdm requests -q

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
DRIVE_BASE = '/content/drive/MyDrive/EMSN-Vocalization'
os.makedirs(f'{DRIVE_BASE}/models', exist_ok=True)
os.makedirs(f'{DRIVE_BASE}/audio', exist_ok=True)
os.makedirs(f'{DRIVE_BASE}/spectrograms', exist_ok=True)
print(f"Drive base: {DRIVE_BASE}")

In [ ]:
# === CONFIGURATIE ===
VERSION = '2025'
EPOCHS = 25
BATCH_SIZE = 32
LEARNING_RATE = 0.001
MIN_SAMPLES = 50  # Minimum per vocalisatie type

# Xeno-canto API key (optioneel maar aanbevolen)
XC_API_KEY = ''  # Vul in als je een API key hebt

In [ ]:
# Ontbrekende soorten voor batch 2
# Format: (Nederlandse naam, Wetenschappelijke naam, directory_naam)

MISSING_SPECIES = [
    # Prioriteit 1 - ZEER BELANGRIJK
    ("Kauw", "Coloeus monedula", "kauw"),
    ("Kokmeeuw", "Chroicocephalus ridibundus", "kokmeeuw"),
    ("Nijlgans", "Alopochen aegyptiaca", "nijlgans"),
    
    # Prioriteit 2 - Regelmatig
    ("Bergeend", "Tadorna tadorna", "bergeend"),
    ("Blauwe Kiekendief", "Circus cyaneus", "blauwe_kiekendief"),
    ("Bonte Strandloper", "Calidris alpina", "bonte_strandloper"),
    ("Boomvalk", "Falco subbuteo", "boomvalk"),
    ("Bosrietzanger", "Acrocephalus palustris", "bosrietzanger"),
    ("Bosruiter", "Tringa glareola", "bosruiter"),
    ("Braamsluiper", "Curruca curruca", "braamsluiper"),
    ("Brilduiker", "Bucephala clangula", "brilduiker"),
    ("Drieteenstrandloper", "Calidris alba", "drieteenstrandloper"),
    ("Eider", "Somateria mollissima", "eider"),
    ("Fluiter", "Phylloscopus sibilatrix", "fluiter"),
    ("Gele Kwikstaart", "Motacilla flava", "gele_kwikstaart"),
    ("Goudplevier", "Pluvialis apricaria", "goudplevier"),
    ("Grasmus", "Curruca communis", "grasmus"),
    ("Groenpootruiter", "Tringa nebularia", "groenpootruiter"),
    ("Grote Gele Kwikstaart", "Motacilla cinerea", "grote_gele_kwikstaart"),
    ("Grote Zaagbek", "Mergus merganser", "grote_zaagbek"),
    ("IJsvogel", "Alcedo atthis", "ijsvogel"),
    ("Kanoetstrandloper", "Calidris canutus", "kanoetstrandloper"),
    ("Keep", "Fringilla montifringilla", "keep"),
    ("Kemphaan", "Calidris pugnax", "kemphaan"),
    ("Kleine Rietgans", "Anser brachyrhynchus", "kleine_rietgans"),
    ("Kleine Strandloper", "Calidris minuta", "kleine_strandloper"),
    ("Kluut", "Recurvirostra avosetta", "kluut"),
    ("Koekoek", "Cuculus canorus", "koekoek"),
    ("Mandarijneend", "Aix galericulata", "mandarijneend"),
    ("Middelste Zaagbek", "Mergus serrator", "middelste_zaagbek"),
    ("Nonnetje", "Mergellus albellus", "nonnetje"),
    ("Oeverloper", "Actitis hypoleucos", "oeverloper"),
    ("Paapje", "Saxicola rubetra", "paapje"),
    ("Pijlstaart", "Anas acuta", "pijlstaart"),
    ("Ransuil", "Asio otus", "ransuil"),
    ("Regenwulp", "Numenius phaeopus", "regenwulp"),
    ("Rietzanger", "Acrocephalus schoenobaenus", "rietzanger"),
    ("Rode Wouw", "Milvus milvus", "rode_wouw"),
    ("Roodhalsfuut", "Podiceps grisegena", "roodhalsfuut"),
    ("Rosse Grutto", "Limosa lapponica", "rosse_grutto"),
    ("Sijs", "Spinus spinus", "sijs"),
    ("Slobeend", "Spatula clypeata", "slobeend"),
    ("Smelleken", "Falco columbarius", "smelleken"),
    ("Steenloper", "Arenaria interpres", "steenloper"),
    ("Tafeleend", "Aythya ferina", "tafeleend"),
    ("Tapuit", "Oenanthe oenanthe", "tapuit"),
    ("Toendrarietgans", "Anser serrirostris", "toendrarietgans"),
    ("Velduil", "Asio flammeus", "velduil"),
    ("Watersnip", "Gallinago gallinago", "watersnip"),
    ("Witgat", "Tringa ochropus", "witgat"),
    ("Zwarte Ruiter", "Tringa erythropus", "zwarte_ruiter"),
    
    # Prioriteit 3 - Minder algemeen (selectie)
    ("Barmsijs", "Acanthis flammea", "barmsijs"),
    ("Beflijster", "Turdus torquatus", "beflijster"),
    ("Bokje", "Lymnocryptes minimus", "bokje"),
    ("Flamingo", "Phoenicopterus roseus", "flamingo"),
    ("Grauwe Kiekendief", "Circus pygargus", "grauwe_kiekendief"),
    ("Grauwe Klauwier", "Lanius collurio", "grauwe_klauwier"),
    ("Klapekster", "Lanius excubitor", "klapekster"),
    ("Kruisbek", "Loxia curvirostra", "kruisbek"),
    ("Oehoe", "Bubo bubo", "oehoe"),
    ("Snor", "Locustella luscinioides", "snor"),
]

print(f"Te trainen: {len(MISSING_SPECIES)} soorten")

In [ ]:
# Xeno-canto API functies
import requests
import time
from pathlib import Path
import json

def search_xeno_canto(scientific_name, voc_type='song', max_results=100):
    """
    Zoek opnames op Xeno-canto.
    voc_type: 'song', 'call', of 'alarm call'
    """
    # Xeno-canto v3 API
    base_url = 'https://xeno-canto.org/api/3/recordings'
    
    params = {
        'query': f'{scientific_name} type:{voc_type} q:A',  # Alleen kwaliteit A
    }
    
    headers = {}
    if XC_API_KEY:
        headers['X-API-KEY'] = XC_API_KEY
    
    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        recordings = data.get('recordings', [])
        return recordings[:max_results]
    except Exception as e:
        print(f"  API error: {e}")
        return []

def download_recording(recording, output_dir):
    """Download een opname van Xeno-canto."""
    xc_id = recording['id']
    file_url = recording.get('file')
    
    if not file_url:
        return None
    
    # Fix URL (soms relatief)
    if file_url.startswith('//'):
        file_url = 'https:' + file_url
    elif not file_url.startswith('http'):
        file_url = 'https://xeno-canto.org' + file_url
    
    output_path = output_dir / f"XC{xc_id}.mp3"
    
    if output_path.exists():
        return output_path
    
    try:
        response = requests.get(file_url, timeout=60)
        response.raise_for_status()
        
        with open(output_path, 'wb') as f:
            f.write(response.content)
        
        return output_path
    except Exception as e:
        print(f"  Download error XC{xc_id}: {e}")
        return None

print("Xeno-canto functies geladen")

In [ ]:
# Spectrogram generatie
import librosa
import numpy as np

SAMPLE_RATE = 48000
N_MELS = 128
N_FFT = 2048
HOP_LENGTH = 512
FMIN = 500
FMAX = 8000
SEGMENT_DURATION = 3.0

def audio_to_spectrograms(audio_path, max_segments=5):
    """Converteer audio naar mel-spectrogrammen."""
    try:
        audio, sr = librosa.load(str(audio_path), sr=SAMPLE_RATE, mono=True)
    except Exception as e:
        return []
    
    segment_samples = int(SEGMENT_DURATION * SAMPLE_RATE)
    spectrograms = []
    
    # Splits in segmenten
    for i in range(0, len(audio), segment_samples):
        if len(spectrograms) >= max_segments:
            break
        
        segment = audio[i:i + segment_samples]
        
        if len(segment) < segment_samples // 2:
            continue
        
        # Pad indien nodig
        if len(segment) < segment_samples:
            segment = np.pad(segment, (0, segment_samples - len(segment)))
        
        # Mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=segment, sr=SAMPLE_RATE,
            n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH,
            fmin=FMIN, fmax=FMAX
        )
        
        # Naar dB en normaliseer
        mel_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_norm = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min() + 1e-8)
        
        # Resize naar 128x128
        if mel_norm.shape != (128, 128):
            from skimage.transform import resize
            mel_norm = resize(mel_norm, (128, 128), anti_aliasing=True)
        
        spectrograms.append(mel_norm)
    
    return spectrograms

print("Spectrogram functies geladen")

In [ ]:
# CNN Model
import torch
import torch.nn as nn

class VocalizationCNN(nn.Module):
    def __init__(self, input_shape=(128, 128), num_classes=3):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
        )
        
        h, w = input_shape[0] // 8, input_shape[1] // 8
        flatten_size = 128 * h * w
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flatten_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
# Complete training pipeline voor één soort
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import time

def train_species_complete(dutch_name, scientific_name, dirname):
    """
    Complete pipeline: download -> spectrograms -> train -> save.
    """
    print(f"\n{'='*60}")
    print(f"Training: {dutch_name} ({scientific_name})")
    print(f"{'='*60}")
    
    start_time = time.time()
    audio_dir = Path(f'{DRIVE_BASE}/audio/{dirname}')
    audio_dir.mkdir(parents=True, exist_ok=True)
    
    X_all, y_all = [], []
    voc_types = [('song', 0), ('call', 1), ('alarm call', 2)]
    
    # Download en verwerk per vocalisatie type
    for voc_type, label in voc_types:
        print(f"  Zoeken: {voc_type}...")
        recordings = search_xeno_canto(scientific_name, voc_type, max_results=30)
        print(f"    Gevonden: {len(recordings)} opnames")
        
        if not recordings:
            continue
        
        type_dir = audio_dir / voc_type.replace(' ', '_')
        type_dir.mkdir(exist_ok=True)
        
        specs_count = 0
        for rec in recordings[:20]:  # Max 20 opnames per type
            audio_path = download_recording(rec, type_dir)
            if audio_path:
                specs = audio_to_spectrograms(audio_path, max_segments=3)
                for spec in specs:
                    X_all.append(spec)
                    y_all.append(label)
                    specs_count += 1
            
            time.sleep(0.5)  # Rate limiting
        
        print(f"    Spectrogrammen: {specs_count}")
    
    # Check of we genoeg data hebben
    X = np.array(X_all)
    y = np.array(y_all)
    
    unique, counts = np.unique(y, return_counts=True)
    print(f"\n  Totaal: {len(X)} spectrogrammen")
    print(f"  Klassen: {dict(zip(['song', 'call', 'alarm'], counts))}")
    
    if len(X) < 50 or len(unique) < 2:
        print(f"  ⚠️ Te weinig data, overslaan")
        return None, 'insufficient_data'
    
    # Train/val split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Naar tensors
    X_train_t = torch.FloatTensor(X_train).unsqueeze(1)
    X_val_t = torch.FloatTensor(X_val).unsqueeze(1)
    y_train_t = torch.LongTensor(y_train)
    y_val_t = torch.LongTensor(y_val)
    
    train_loader = DataLoader(
        TensorDataset(X_train_t, y_train_t),
        batch_size=BATCH_SIZE, shuffle=True
    )
    val_loader = DataLoader(
        TensorDataset(X_val_t, y_val_t),
        batch_size=BATCH_SIZE
    )
    
    # Model
    model = VocalizationCNN(num_classes=len(unique)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Training
    best_acc = 0
    
    for epoch in range(EPOCHS):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        
        # Validate
        model.eval()
        val_correct = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                val_correct += (outputs.argmax(1) == y_batch).sum().item()
        
        val_acc = val_correct / len(y_val)
        if val_acc > best_acc:
            best_acc = val_acc
        
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{EPOCHS} - val_acc: {val_acc:.1%}")
    
    # Save model
    model_path = Path(f'{DRIVE_BASE}/models/{dirname}_cnn_{VERSION}.pt')
    torch.save({
        'model_state_dict': model.state_dict(),
        'num_classes': len(unique),
        'accuracy': best_acc,
        'species_name': dutch_name,
        'scientific_name': scientific_name,
        'version': VERSION
    }, model_path)
    
    elapsed = time.time() - start_time
    print(f"\n  ✅ Model opgeslagen: {model_path.name}")
    print(f"  Accuracy: {best_acc:.1%}")
    print(f"  Tijd: {elapsed/60:.1f} minuten")
    
    return best_acc, 'success'

In [ ]:
# TRAIN ALLE ONTBREKENDE SOORTEN
from datetime import datetime

results = []
start_all = time.time()

print(f"Start: {datetime.now()}")
print(f"Soorten: {len(MISSING_SPECIES)}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print()

for i, (dutch, scientific, dirname) in enumerate(MISSING_SPECIES):
    print(f"\n[{i+1}/{len(MISSING_SPECIES)}]", end='')
    
    try:
        acc, status = train_species_complete(dutch, scientific, dirname)
        results.append({
            'species': dutch,
            'scientific': scientific,
            'accuracy': acc,
            'status': status
        })
    except Exception as e:
        print(f"  ❌ Error: {e}")
        results.append({
            'species': dutch,
            'scientific': scientific,
            'accuracy': None,
            'status': f'error: {str(e)[:50]}'
        })
    
    # Tussentijds opslaan
    if (i + 1) % 10 == 0:
        import pandas as pd
        pd.DataFrame(results).to_csv(
            f'{DRIVE_BASE}/training_results_batch2_checkpoint.csv',
            index=False
        )
        print(f"\n  📁 Checkpoint opgeslagen")

elapsed_all = time.time() - start_all
print(f"\n{'='*60}")
print(f"KLAAR!")
print(f"Totale tijd: {elapsed_all/3600:.1f} uur")
print(f"Succesvol: {sum(1 for r in results if r['status'] == 'success')}")
print(f"Mislukt: {sum(1 for r in results if r['status'] != 'success')}")

In [ ]:
# Resultaten opslaan
import pandas as pd

df = pd.DataFrame(results)
df.to_csv(f'{DRIVE_BASE}/training_results_batch2_{VERSION}.csv', index=False)

successful = df[df['status'] == 'success']
print(f"\nSamenvatting:")
print(f"  Getraind: {len(successful)} soorten")
if len(successful) > 0:
    print(f"  Gem. accuracy: {successful['accuracy'].mean():.1%}")
    print(f"  Min accuracy: {successful['accuracy'].min():.1%}")
    print(f"  Max accuracy: {successful['accuracy'].max():.1%}")
    print(f"\nTop 10:")
    print(successful.nlargest(10, 'accuracy')[['species', 'accuracy']].to_string(index=False))

In [ ]:
# Check hoeveel modellen er nu zijn
models_dir = Path(f'{DRIVE_BASE}/models')
models = list(models_dir.glob('*.pt'))
print(f"\nModellen in Drive: {len(models)}")
print(f"\nGebruik rclone om te syncen naar je Pi:")
print(f"  rclone sync gdrive:EMSN-Vocalization/models/ ~/emsn-vocalization/trained-models/ -P")